In [39]:
# https://docs.bittensor.com/subnets/register-validate-mine#check-the-permit-status
"""
Except in Subnet 1, all subnets have 256 UID slots per subnet. Of these 256 UID slots,
a subnet can have a maximum of 64 subnet validator UIDs and 192 (i.e., 256-64) UIDs for subnet miners.
"""

import bittensor as bt
import asyncio

# NET_UID = 61
NET_UID = 170
NETWORK = "test"
WALLET = "validator"

In [40]:
# Initialize your wallet
wallet = bt.wallet(name=WALLET)

# Initialize the subtensor
subtensor = bt.subtensor(network=NETWORK)

# Fetch the metagraph
subnet = subtensor.metagraph(netuid=NET_UID)

# Replace below with your SS58 hotkey 
hotkey = wallet.hotkey.ss58_address
my_uid = subnet.hotkeys.index(wallet.hotkey.ss58_address)
sub = bt.subtensor(NETWORK)
mg = sub.metagraph(NET_UID)
if hotkey not in mg.hotkeys:
    print(f"Hotkey {hotkey} deregistered")
else:
    print(f"Hotkey {hotkey} is registered. UID: {my_uid}")

INFO:bittensor: - Connected to test network and wss://test.finney.opentensor.ai:443/. - 
INFO:bittensor: - Connected to test network and wss://test.finney.opentensor.ai:443/. - 


Hotkey 5Fk35HgrTqqUffK7WN8FG4euZ8MpKx35mUYz9kgwj3UDnNHr is registered. UID: 0


In [41]:
# Extract active UIDs
active_uids = subnet.uids.tolist()

# Print the active UIDs
print(f"Active UIDs (total: {len(active_uids)}): {active_uids}")
print(f"My hotkey is active: {my_uid in active_uids}")
print(f"Is val permit: {subnet.validator_permit[my_uid]}")
print(f"1000+ TAO:")

Active UIDs (total: 2): [0, 1]
My hotkey is active: True
Is val permit: True
1000+ TAO:


In [42]:
# Sample validators.
# Define your request payload (e.g., a text prompting request)
# import time
from prompting.protocol import StreamPromptingSynapse
# request_payload = bt.synapse.TextPrompting(
#     text="Hello, can you process this request?"
# )
# request_payload

# Get the list of uids to query for this step.
# uids = get_random_uids(self, k=k, exclude=exclude or []).to(self.device)
# uids = get_random_uids(self, k=k, exclude=exclude or []).to(self.device)
# uids_cpu = uids.cpu().tolist()

uids = [my_uid]
axons = [subnet.axons[uid] for uid in uids]

# # Directly call dendrite and process responses in parallel
dendrite = bt.dendrite(wallet=wallet)
roles = ["user"]
messages = ["Organic prompt query. Question: Capital of Zimbabwe?"]
timeout = 40
streams_responses = await dendrite(
    axons=axons,
    synapse=StreamPromptingSynapse(roles=roles, messages=messages),
    timeout=timeout,
    deserialize=False,
    streaming=True,
)
# Prepare the task for handling stream responses
from prompting.dendrite import DendriteResponseEvent
from prompting.forward import handle_response
from transformers import AutoTokenizer

stream_results_dict = dict(zip(uids, streams_responses))
model_name = "casperhansen/llama-3-8b-instruct-awq"
tokenizer = AutoTokenizer.from_pretrained(model_name)
handle_stream_responses_task = asyncio.create_task(handle_response(stream_results_dict, tokenizer))
stream_results = await handle_stream_responses_task
response_event = DendriteResponseEvent(
    stream_results=stream_results, uids=uids, timeout=timeout
)

bt.logging.info(f"Created DendriteResponseEvent:\n {response_event}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
INFO:bittensor: - Created DendriteResponseEvent:
 DendriteResponseEvent(uids=[0], completions=['{"name":"StreamPromptingSynapse","timeout":40.0,"total_size":4036,"header_size":0,"dendrite":{"status_code":null,"status_message":null,"process_time":null,"ip":"194.68.245.74","port":null,"version":701,"nonce":16748188472635226,"uuid":"6b47886a-2e1d-11ef-9141-0242c0a80902","hotkey":"5Fk35HgrTqqUffK7WN8FG4euZ8MpKx35mUYz9kgwj3UDnNHr","signature":"0x0003bf86a3f219bca84df126263784401d524d846ea3834c7e88e21fcb676137fd8de8c3990a5e27e42211e5d5146175eeea06beb4712b7f040d6486e831b58f"},"axon":{"status_code":null,"status_message":null,"process_time":null,"ip":"194.68.245.74","port":8091,"version":null,"nonce":null,"uuid":null,"hotkey":"5Fk35HgrTqqUffK7WN8FG4euZ8MpKx35mUYz9kgwj3UDnNHr","signature":null},"computed_body_hash":"","required_hash_fields":["messages"],"roles":["user"],"messages

In [53]:
stream_results

[SynapseStreamResult(exception=UnboundLocalError("local variable 'chunk' referenced before assignment"), uid=166, accumulated_chunks=[], accumulated_chunks_timings=[], tokens_per_chunk=[], synapse=StreamPromptingSynapse(required_hash_fields=['messages'], roles=['user'], messages=['failure'], completion=''))]

In [44]:
response_event

DendriteResponseEvent(uids=[166], completions=[''], timings=[0], status_messages=['Service at 194.68.245.74:8091/StreamPromptingSynapse unavailable.'], status_codes=[503])

In [35]:
streams_responses

[<async_generator object DendriteMixin.call_stream at 0x7f2dc3d30fc0>]